In [0]:
import pandas as pd


from pylift import TransformedOutcome
import numpy as np

from collections import Counter
from sklearn.preprocessing import MinMaxScaler
import math



import matplotlib.pyplot as plt
import subprocess
import sys

from pylift import TransformedOutcome
from pylift.eval import UpliftEval
import random
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from collections import Counter

from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
from sklearn.metrics.cluster import contingency_matrix
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

In [0]:
pd.options.mode.chained_assignment = None

In [0]:
%matplotlib inline

In [0]:
df = pd.read_csv('~/Downloads/uplift_datasets/criteo-uplift-v2.1.csv.gz')
print(df.shape)
print('\ntreatment distribition:')
print(str(df.treatment.value_counts(normalize=True)))

(13979592, 16)

treatment distribition:
1 0.85
0 0.15
Name: treatment, dtype: float64

In [0]:
df.groupby(['treatment', 'visit']).agg({'visit':[ 'count']})

visit 
 
 
 
 
 count 
 
 
 treatment 
 visit 
 
 
 
 
 
 0 
 0 
 2016832 
 
 
 1 
 80105 
 
 
 1 
 0 
 11305831 
 
 
 1 
 576824

In [0]:
df.groupby(['treatment', 'conversion']).agg({'conversion':[ 'count']})

conversion 
 
 
 
 
 count 
 
 
 treatment 
 conversion 
 
 
 
 
 
 0 
 0 
 2092874 
 
 
 1 
 4063 
 
 
 1 
 0 
 11845944 
 
 
 1 
 36711

In [0]:
features = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11']

In [0]:
df_sample = df.sample(frac =.5) 
df_sample.shape

Out[8]: (6989796, 16)

In [0]:
no_of_splits_kfold = 5

In [0]:
outcomeCol   = 'visit'
treatmentCol = 'treatment'

TMA_VisitTreatmentMts = {}
TMA_VisitTreatmentMts['q2_qini']   = []
TMA_VisitTreatmentMts['q2_aqini']  = []
TMA_VisitTreatmentMts['q2_cgains'] = []

kf = KFold(n_splits=no_of_splits_kfold)
for train_index, test_index in kf.split(df_sample):
    train_df = df_sample.iloc[train_index]
    test_df  = df_sample.iloc[test_index]
    
    treated_data    = train_df[train_df[treatmentCol] == True ]
    control_data    = train_df[train_df[treatmentCol] == False] 

    P_T = GradientBoostingClassifier().fit(treated_data[features], treated_data[outcomeCol] )
    P_C = GradientBoostingClassifier().fit(control_data[features], control_data[outcomeCol] )

    test_data_val_pred    = P_T.predict_proba( test_df[features] ) 
    control_data_val_pred = P_C.predict_proba( test_df[features] )

    upev = UpliftEval(test_df[treatmentCol], test_df[outcomeCol], ( test_data_val_pred[:,1] - control_data_val_pred[:,1] ) )

    print('q2_aqini: ', round(upev.q2_aqini,3), 'q2_qini: ', round(upev.q2_qini,3), 'q2_cgains: ', round(upev.q2_cgains,3))
    
    TMA_VisitTreatmentMts['q2_qini'].append(upev.q2_qini) 
    TMA_VisitTreatmentMts['q2_aqini'].append(upev.q2_aqini)
    TMA_VisitTreatmentMts['q2_cgains'].append(upev.q2_cgains)
     
#     upev.plot(plot_type='cgains', label='Model: TMA; TV=treatment & OV=visit')

q2_aqini: 0.096 q2_qini: 0.116 q2_cgains: 0.093
q2_aqini: 0.081 q2_qini: 0.1 q2_cgains: 0.079
q2_aqini: 0.084 q2_qini: 0.102 q2_cgains: 0.081
q2_aqini: 0.082 q2_qini: 0.1 q2_cgains: 0.08
q2_aqini: 0.086 q2_qini: 0.105 q2_cgains: 0.083

In [0]:
outcomeCol   = 'conversion'
treatmentCol = 'treatment'

TMA_ConversionTreatmentMts = {}
TMA_ConversionTreatmentMts['q2_qini']   = []
TMA_ConversionTreatmentMts['q2_aqini']  = []
TMA_ConversionTreatmentMts['q2_cgains'] = []

kf = KFold(n_splits=no_of_splits_kfold)
for train_index, test_index in kf.split(df_sample):
    train_df = df_sample.iloc[train_index]
    test_df  = df_sample.iloc[test_index]
    
    treated_data    = train_df[train_df[treatmentCol] == True ]
    control_data    = train_df[train_df[treatmentCol] == False] 

    P_T = GradientBoostingClassifier().fit(treated_data[features], treated_data[outcomeCol] )
    P_C = GradientBoostingClassifier().fit(control_data[features], control_data[outcomeCol] )

    test_data_val_pred    = P_T.predict_proba( test_df[features] ) 
    control_data_val_pred = P_C.predict_proba( test_df[features] )

    upev = UpliftEval(test_df[treatmentCol], test_df[outcomeCol], ( test_data_val_pred[:,1] - control_data_val_pred[:,1] ) )

    print('q2_aqini: ', round(upev.q2_aqini,3), 'q2_qini: ', round(upev.q2_qini,3), 'q2_cgains: ', round(upev.q2_cgains,3))
    
    TMA_ConversionTreatmentMts['q2_qini'].append(upev.q2_qini) 
    TMA_ConversionTreatmentMts['q2_aqini'].append(upev.q2_aqini)
    TMA_ConversionTreatmentMts['q2_cgains'].append(upev.q2_cgains)
     
#     upev.plot(plot_type='cgains', label='Model: TMA; TV=treatment & OV=visit')

q2_aqini: 0.132 q2_qini: 0.149 q2_cgains: 0.127
q2_aqini: 0.077 q2_qini: 0.091 q2_cgains: 0.073
q2_aqini: 0.108 q2_qini: 0.127 q2_cgains: 0.103
q2_aqini: 0.137 q2_qini: 0.151 q2_cgains: 0.131
q2_aqini: 0.153 q2_qini: 0.167 q2_cgains: 0.148

In [0]:
outcomeCol   = 'visit'
treatmentCol = 'treatment'

SMA_VisitTreatmentMts = {}
SMA_VisitTreatmentMts['q2_qini']   = []
SMA_VisitTreatmentMts['q2_aqini']  = []
SMA_VisitTreatmentMts['q2_cgains'] = []

kf = KFold(n_splits=no_of_splits_kfold)
for train_index, test_index in kf.split(df_sample):
    
    train_df = df_sample.iloc[train_index]
    test_df  = df_sample.iloc[test_index]
    
    train_df[f'{treatmentCol}_mod'] = train_df[treatmentCol]
    test_df[f'{treatmentCol}_mod']  = test_df[treatmentCol]

    propModel = GradientBoostingClassifier().fit(train_df[features + [f'{treatmentCol}_mod']], train_df[outcomeCol] )

    test_df[f'{treatmentCol}_mod'] = 1
    test_data_val_pred    = propModel.predict_proba( test_df[features + [f'{treatmentCol}_mod']] )
    test_df[f'{treatmentCol}_mod'] = 0
    control_data_val_pred = propModel.predict_proba( test_df[features + [f'{treatmentCol}_mod']] )

    upev = UpliftEval(test_df[treatmentCol], test_df[outcomeCol], ( test_data_val_pred[:,1] - control_data_val_pred[:,1] ) )

    SMA_VisitTreatmentMts['q2_qini'].append(upev.q2_qini) 
    SMA_VisitTreatmentMts['q2_aqini'].append(upev.q2_aqini)
    SMA_VisitTreatmentMts['q2_cgains'].append(upev.q2_cgains)
    
    print('q2_aqini: ', round(upev.q2_aqini,3), 'q2_qini: ', round(upev.q2_qini,3), 'q2_cgains: ', round(upev.q2_cgains,3))
#     upev.plot(plot_type='cgains', label='Model: SMA; TV=treatment & OV=visit')


q2_aqini: 0.088 q2_qini: 0.112 q2_cgains: 0.085
q2_aqini: 0.085 q2_qini: 0.109 q2_cgains: 0.083
q2_aqini: 0.083 q2_qini: 0.106 q2_cgains: 0.081
q2_aqini: 0.091 q2_qini: 0.114 q2_cgains: 0.088
q2_aqini: 0.083 q2_qini: 0.106 q2_cgains: 0.081

In [0]:
outcomeCol   = 'conversion'
treatmentCol = 'treatment'

SMA_ConversionTreatmentMts = {}
SMA_ConversionTreatmentMts['q2_qini']   = []
SMA_ConversionTreatmentMts['q2_aqini']  = []
SMA_ConversionTreatmentMts['q2_cgains'] = []

kf = KFold(n_splits=no_of_splits_kfold)
for train_index, test_index in kf.split(df_sample):
    
    train_df = df_sample.iloc[train_index]
    test_df  = df_sample.iloc[test_index]
    
    train_df[f'{treatmentCol}_mod'] = train_df[treatmentCol]
    test_df[f'{treatmentCol}_mod']  = test_df[treatmentCol]

    propModel = GradientBoostingClassifier().fit(train_df[features + [f'{treatmentCol}_mod']], train_df[outcomeCol] )


    test_df[f'{treatmentCol}_mod'] = 1
    test_data_val_pred    = propModel.predict_proba( test_df[features + [f'{treatmentCol}_mod']] )
    test_df[f'{treatmentCol}_mod'] = 0
    control_data_val_pred = propModel.predict_proba( test_df[features + [f'{treatmentCol}_mod']] )

    upev = UpliftEval(test_df[treatmentCol], test_df[outcomeCol], ( test_data_val_pred[:,1] - control_data_val_pred[:,1] ) )

    SMA_ConversionTreatmentMts['q2_qini'].append(upev.q2_qini) 
    SMA_ConversionTreatmentMts['q2_aqini'].append(upev.q2_aqini)
    SMA_ConversionTreatmentMts['q2_cgains'].append(upev.q2_cgains)
    
    print('q2_aqini: ', round(upev.q2_aqini,3), 'q2_qini: ', round(upev.q2_qini,3), 'q2_cgains: ', round(upev.q2_cgains,3))
#     upev.plot(plot_type='cgains', label='Model: SMA; TV=treatment & OV=visit')


q2_aqini: 0.209 q2_qini: 0.229 q2_cgains: 0.203
q2_aqini: 0.016 q2_qini: 0.016 q2_cgains: 0.016
q2_aqini: 0.006 q2_qini: 0.007 q2_cgains: 0.005
q2_aqini: 0.008 q2_qini: 0.007 q2_cgains: 0.008
q2_aqini: 0.01 q2_qini: 0.01 q2_cgains: 0.01

In [0]:
outcomeCol   = 'visit'
treatmentCol = 'treatment'

OT_VisitTreatmentMts = {}
OT_VisitTreatmentMts['q2_qini']   = []
OT_VisitTreatmentMts['q2_aqini']  = []
OT_VisitTreatmentMts['q2_cgains'] = []

kf = KFold(n_splits=no_of_splits_kfold)
for train_index, test_index in kf.split(df_sample):    
    train_df = df_sample.iloc[train_index]
    test_df  = df_sample.iloc[test_index]
    
    psModel = GradientBoostingClassifier().fit( train_df[features], train_df[treatmentCol]  )    
    propensityScore = psModel.predict_proba( train_df[features] )    
    train_df['ps'] = np.where(train_df[treatmentCol] == 1, propensityScore[:,1], propensityScore[:,0])
    train_df['y_mod']  = ( 
                            train_df[outcomeCol] * ( train_df[treatmentCol] - train_df['ps']) 
                                       / 
                           ( train_df['ps'] * ( 1-train_df['ps'] ) )
                          )
    cateModel = GradientBoostingRegressor().fit(train_df[features], train_df['y_mod'] )

    test_df['uplift'] = cateModel.predict( test_df[features] )
    upev = UpliftEval(test_df[treatmentCol], test_df[outcomeCol], test_df.uplift)
    
    OT_VisitTreatmentMts['q2_qini'].append(upev.q2_qini) 
    OT_VisitTreatmentMts['q2_aqini'].append(upev.q2_aqini)
    OT_VisitTreatmentMts['q2_cgains'].append(upev.q2_cgains)
    
    print('q2_aqini: ', round(upev.q2_aqini,3), 'q2_qini: ', round(upev.q2_qini,3), 'q2_cgains: ', round(upev.q2_cgains,3))
#     upev.plot(plot_type='cgains', label='Model: OT; TV=treatment & OV=visit')

q2_aqini: 0.086 q2_qini: 0.108 q2_cgains: 0.084
q2_aqini: 0.081 q2_qini: 0.103 q2_cgains: 0.079
q2_aqini: 0.083 q2_qini: 0.103 q2_cgains: 0.081
q2_aqini: 0.088 q2_qini: 0.109 q2_cgains: 0.086
q2_aqini: 0.081 q2_qini: 0.102 q2_cgains: 0.079

In [0]:
outcomeCol   = 'conversion'
treatmentCol = 'treatment'

OT_ConversionTreatmentMts = {}
OT_ConversionTreatmentMts['q2_qini']   = []
OT_ConversionTreatmentMts['q2_aqini']  = []
OT_ConversionTreatmentMts['q2_cgains'] = []

kf = KFold(n_splits=no_of_splits_kfold)
for train_index, test_index in kf.split(df_sample):    
    train_df = df_sample.iloc[train_index]
    test_df  = df_sample.iloc[test_index]
    
    psModel = GradientBoostingClassifier().fit( train_df[features], train_df[treatmentCol]  )    
    propensityScore = psModel.predict_proba( train_df[features] )    
    train_df['ps'] = np.where(train_df[treatmentCol] == 1, propensityScore[:,1], propensityScore[:,0])
    train_df['y_mod']  = ( 
                            train_df[outcomeCol] * ( train_df[treatmentCol] - train_df['ps']) 
                                       / 
                           ( train_df['ps'] * ( 1-train_df['ps'] ) )
                          )
    cateModel = GradientBoostingRegressor().fit(train_df[features], train_df['y_mod'] )

    test_df['uplift'] = cateModel.predict( test_df[features] )
    upev = UpliftEval(test_df[treatmentCol], test_df[outcomeCol], test_df.uplift)
    
    OT_ConversionTreatmentMts['q2_qini'].append(upev.q2_qini) 
    OT_ConversionTreatmentMts['q2_aqini'].append(upev.q2_aqini)
    OT_ConversionTreatmentMts['q2_cgains'].append(upev.q2_cgains)
    
    print('q2_aqini: ', round(upev.q2_aqini,3), 'q2_qini: ', round(upev.q2_qini,3), 'q2_cgains: ', round(upev.q2_cgains,3))
#     upev.plot(plot_type='cgains', label='Model: OT; TV=treatment & OV=visit')

q2_aqini: 0.213 q2_qini: 0.235 q2_cgains: 0.207
q2_aqini: 0.17 q2_qini: 0.194 q2_cgains: 0.165
q2_aqini: 0.166 q2_qini: 0.189 q2_cgains: 0.161
q2_aqini: 0.195 q2_qini: 0.218 q2_cgains: 0.189
q2_aqini: 0.188 q2_qini: 0.211 q2_cgains: 0.182

In [0]:
outcomeCol   = 'visit'
treatmentCol = 'treatment'

XL_VisitTreatmentMts = {}
XL_VisitTreatmentMts['q2_qini']   = []
XL_VisitTreatmentMts['q2_aqini']  = []
XL_VisitTreatmentMts['q2_cgains'] = []

kf = KFold(n_splits=no_of_splits_kfold)
for train_index, test_index in kf.split(df_sample):    
    train_df = df_sample.iloc[train_index]
    test_df  = df_sample.iloc[test_index]


    psModel = GradientBoostingClassifier().fit( train_df[features], train_df[treatmentCol]  )

    treated_data    = train_df[train_df.treatment == True ]
    control_data    = train_df[train_df.treatment == False] 

    T_X = GradientBoostingClassifier().fit(treated_data[features], treated_data[outcomeCol] )
    C_X = GradientBoostingClassifier().fit(control_data[features], control_data[outcomeCol] )

    treated_data['DT_Y'] = treated_data[outcomeCol] - C_X.predict_proba(treated_data[features])[:,1]
    control_data['DC_Y'] = T_X.predict_proba(control_data[features])[:,1] - control_data[outcomeCol]

    DT_X = GradientBoostingRegressor().fit(treated_data[features], treated_data['DT_Y'])
    DC_X = GradientBoostingRegressor().fit(control_data[features], control_data['DC_Y'])

    test_df['uplift'] = ( 
        psModel.predict_proba(test_df[features])[:,1] * DC_X.predict(test_df[features])
                             +
        psModel.predict_proba(test_df[features])[:,0] * DT_X.predict(test_df[features])
    )

    upev = UpliftEval(test_df[treatmentCol], test_df[outcomeCol], test_df.uplift)

    print('q2_aqini: ', round(upev.q2_aqini,3), 'q2_qini: ', round(upev.q2_qini,3), 'q2_cgains: ', round(upev.q2_cgains,3))
#     upev.plot(plot_type='cgains', label='Model: X-Learner; TV=treatment & OV=visit')

    XL_VisitTreatmentMts['q2_qini'].append(upev.q2_qini) 
    XL_VisitTreatmentMts['q2_aqini'].append(upev.q2_aqini)
    XL_VisitTreatmentMts['q2_cgains'].append(upev.q2_cgains)
    


q2_aqini: 0.09 q2_qini: 0.111 q2_cgains: 0.087
q2_aqini: 0.085 q2_qini: 0.106 q2_cgains: 0.082
q2_aqini: 0.09 q2_qini: 0.108 q2_cgains: 0.088
q2_aqini: 0.094 q2_qini: 0.114 q2_cgains: 0.091
q2_aqini: 0.087 q2_qini: 0.107 q2_cgains: 0.084

In [0]:
outcomeCol   = 'conversion'
treatmentCol = 'treatment'

XL_ConversionTreatmentMts = {}
XL_ConversionTreatmentMts['q2_qini']   = []
XL_ConversionTreatmentMts['q2_aqini']  = []
XL_ConversionTreatmentMts['q2_cgains'] = []

kf = KFold(n_splits=no_of_splits_kfold)
for train_index, test_index in kf.split(df_sample):    
    train_df = df_sample.iloc[train_index]
    test_df  = df_sample.iloc[test_index]


    psModel = GradientBoostingClassifier().fit( train_df[features], train_df[treatmentCol]  )

    treated_data    = train_df[train_df.treatment == True ]
    control_data    = train_df[train_df.treatment == False] 

    T_X = GradientBoostingClassifier().fit(treated_data[features], treated_data[outcomeCol] )
    C_X = GradientBoostingClassifier().fit(control_data[features], control_data[outcomeCol] )

    treated_data['DT_Y'] = treated_data[outcomeCol] - C_X.predict_proba(treated_data[features])[:,1]
    control_data['DC_Y'] = T_X.predict_proba(control_data[features])[:,1] - control_data[outcomeCol]

    DT_X = GradientBoostingRegressor().fit(treated_data[features], treated_data['DT_Y'])
    DC_X = GradientBoostingRegressor().fit(control_data[features], control_data['DC_Y'])

    test_df['uplift'] = ( 
        psModel.predict_proba(test_df[features])[:,1] * DC_X.predict(test_df[features])
                             +
        psModel.predict_proba(test_df[features])[:,0] * DT_X.predict(test_df[features])
    )

    upev = UpliftEval(test_df[treatmentCol], test_df[outcomeCol], test_df.uplift)

    print('q2_aqini: ', round(upev.q2_aqini,3), 'q2_qini: ', round(upev.q2_qini,3), 'q2_cgains: ', round(upev.q2_cgains,3))
#     upev.plot(plot_type='cgains', label='Model: X-Learner; TV=treatment & OV=visit')

    XL_ConversionTreatmentMts['q2_qini'].append(upev.q2_qini) 
    XL_ConversionTreatmentMts['q2_aqini'].append(upev.q2_aqini)
    XL_ConversionTreatmentMts['q2_cgains'].append(upev.q2_cgains)
    


q2_aqini: 0.19 q2_qini: 0.21 q2_cgains: 0.184
q2_aqini: 0.095 q2_qini: 0.112 q2_cgains: 0.091
q2_aqini: 0.154 q2_qini: 0.174 q2_cgains: 0.149
q2_aqini: 0.164 q2_qini: 0.181 q2_cgains: 0.158
q2_aqini: 0.18 q2_qini: 0.197 q2_cgains: 0.175

In [0]:
ModelMetrics = [
            ('TMA: visit       + treatment', TMA_VisitTreatmentMts), 
            ('TMA: conversion  + treatment', TMA_ConversionTreatmentMts), 
            ('SMA: visit       + treatment', SMA_VisitTreatmentMts), 
            ('SMA: conversion  + treatment', SMA_ConversionTreatmentMts), 
            ('OT:  visit       + treatment', OT_VisitTreatmentMts), 
            ('OT:  conversion  + treatment', OT_ConversionTreatmentMts), 
            ('XL:  visit       + treatment', XL_VisitTreatmentMts), 
            ('XL:  conversion  + treatment', XL_ConversionTreatmentMts)
]
for (modelName, modelMetrics) in ModelMetrics:
    print(modelName)
    print( '\t mean: ', round(np.mean(modelMetrics['q2_qini']),  3),   'std: ', round(np.std(modelMetrics['q2_aqini']) ,3) )
    print( '\t mean: ', round(np.mean(modelMetrics['q2_aqini']), 3),   'std: ', round(np.std(modelMetrics['q2_aqini']) ,3) )
    print( '\t mean: ', round(np.mean(modelMetrics['q2_cgains']),3),   'std: ', round(np.std(modelMetrics['q2_cgains']),3) )
    print('\n')


TMA: visit + treatment
 mean: 0.105 std: 0.005
 mean: 0.086 std: 0.005
 mean: 0.083 std: 0.005


TMA: conversion + treatment
 mean: 0.137 std: 0.026
 mean: 0.121 std: 0.026
 mean: 0.116 std: 0.026


SMA: visit + treatment
 mean: 0.109 std: 0.003
 mean: 0.086 std: 0.003
 mean: 0.084 std: 0.003


SMA: conversion + treatment
 mean: 0.054 std: 0.08
 mean: 0.05 std: 0.08
 mean: 0.049 std: 0.078


OT: visit + treatment
 mean: 0.105 std: 0.003
 mean: 0.084 std: 0.003
 mean: 0.082 std: 0.003


OT: conversion + treatment
 mean: 0.209 std: 0.017
 mean: 0.186 std: 0.017
 mean: 0.181 std: 0.017


XL: visit + treatment
 mean: 0.109 std: 0.003
 mean: 0.089 std: 0.003
 mean: 0.086 std: 0.003


XL: conversion + treatment
 mean: 0.175 std: 0.033
 mean: 0.157 std: 0.033
 mean: 0.151 std: 0.033

In [0]:
ModelMetrics = [
            ('TMA: visit       + treatment', TMA_VisitTreatmentMts), 
            ('TMA: conversion  + treatment', TMA_ConversionTreatmentMts), 
            ('SMA: visit       + treatment', SMA_VisitTreatmentMts), 
            ('SMA: conversion  + treatment', SMA_ConversionTreatmentMts), 
            ('OT:  visit       + treatment', OT_VisitTreatmentMts), 
            ('OT:  conversion  + treatment', OT_ConversionTreatmentMts), 
            ('XL:  visit       + treatment', XL_VisitTreatmentMts), 
            ('XL:  conversion  + treatment', XL_ConversionTreatmentMts)
]
for (modelName, modelMetrics) in ModelMetrics:
    print(modelName)
    print('\t', modelMetrics['q2_qini'])
    print('\t', modelMetrics['q2_aqini'])
    print('\t', modelMetrics['q2_cgains'])

TMA: visit + treatment
 [0.11617325871812029, 0.1004508201409335, 0.10157777310526447, 0.09965827250525336, 0.10504298819923442]
 [0.09647550575365084, 0.08134322670786229, 0.08397233769815406, 0.08226609320069926, 0.08617335873273574]
 [0.09324649044285886, 0.07861948282947732, 0.08133872126772519, 0.07966335186663229, 0.08342339661107713]
TMA: conversion + treatment
 [0.1486189778942153, 0.09097950271034998, 0.12727195609685754, 0.15083308134501014, 0.1673643197424234]
 [0.13175862892748108, 0.07711996462609573, 0.10795862236898646, 0.1365086356644537, 0.15287656683813897]
 [0.12657899415106474, 0.0728151750288048, 0.10338156422212409, 0.13096248588936013, 0.14772992663780554]
SMA: visit + treatment
 [0.1119931274924665, 0.10869849973697425, 0.10617960134904351, 0.11373597007326888, 0.10578060871941869]
 [0.08827979747471101, 0.08545254969028453, 0.08349459872085696, 0.09062204286041144, 0.08322194482354092]
 [0.08545035452512946, 0.08275676617114736, 0.08092871896945504, 0.08771628744110495, 0.08076038273519026]
SMA: conversion + treatment
 [0.22897388623595555, 0.01616976592160363, 0.007223430336616054, 0.00744350251581592, 0.01008050338246231]
 [0.20940182243714922, 0.016412729165855365, 0.005789009031663772, 0.007621539312816839, 0.009954501326890014]
 [0.20347207163825706, 0.016487656752357367, 0.0054205474032254265, 0.007668447573298726, 0.009879794777420766]
OT: visit + treatment
 [0.10766675984876489, 0.10325751822438345, 0.10258877286006735, 0.10871787658833339, 0.10172912376764223]
 [0.08599325055218308, 0.08122896527052156, 0.08300206647130438, 0.08781378221753915, 0.08079553423645727]
 [0.08384626031279008, 0.07917044527033228, 0.08114858380511518, 0.08572005805078099, 0.0788656100650473]
OT: conversion + treatment
 [0.2352629300534, 0.19359893245030751, 0.18922035305610466, 0.21774198201207492, 0.21051579186740307]
 [0.21315564358836522, 0.16977100524135824, 0.16569073912099277, 0.19516125090548317, 0.18782466848740406]
 [0.2066090612414756, 0.16454445964401362, 0.1607861368908862, 0.18929760937650705, 0.18228651060144052]
XL: visit + treatment
 [0.11051823224332372, 0.10615069408940515, 0.10815567581458387, 0.11380392941522635, 0.10712815029697334]
 [0.08997357005297671, 0.08451024640609421, 0.09019161497426625, 0.09433293803115682, 0.08659555969072234]
 [0.08682494520600513, 0.08153927719125374, 0.08755386143531825, 0.0913528037061221, 0.08380262915551702]
XL: conversion + treatment
 [0.2100918104574187, 0.1123116835296516, 0.17393054323183868, 0.18142383992041805, 0.19727042683448767]
 [0.1904640090200632, 0.09496569984445503, 0.1536168212899018, 0.16367916067619756, 0.18036800275363202]
 [0.18440018545544232, 0.09066568802607163, 0.1488742519376174, 0.1580331692606642, 0.17513773346353181]

In [0]:
# TMA: visit       + treatment
# 	 mean:  0.105 std:  0.007
# 	 mean:  0.086 std:  0.007
# 	 mean:  0.083 std:  0.007


# TMA: conversion  + treatment
# 	 mean:  0.145 std:  0.035
# 	 mean:  0.127 std:  0.035
# 	 mean:  0.123 std:  0.034


# SMA: visit       + treatment
# 	 mean:  0.108 std:  0.004
# 	 mean:  0.083 std:  0.004
# 	 mean:  0.08 std:  0.004


# SMA: conversion  + treatment
# 	 mean:  0.142 std:  0.064
# 	 mean:  0.129 std:  0.064
# 	 mean:  0.126 std:  0.062


# OT:  visit       + treatment
# 	 mean:  0.103 std:  0.003
# 	 mean:  0.082 std:  0.003
# 	 mean:  0.081 std:  0.003


# OT:  conversion  + treatment
# 	 mean:  0.208 std:  0.016
# 	 mean:  0.185 std:  0.016
# 	 mean:  0.18 std:  0.015


# XL:  visit       + treatment
# 	 mean:  0.11 std:  0.007
# 	 mean:  0.091 std:  0.007
# 	 mean:  0.087 std:  0.007


# XL:  conversion  + treatment
# 	 mean:  0.176 std:  0.03
# 	 mean:  0.155 std:  0.03
# 	 mean:  0.149 std:  0.029

